<a href="https://colab.research.google.com/github/liriacs/analise_tecnica/blob/main/AT_efic_candles_engolf_harami_janela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance 
import yfinance as yf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates

In [3]:
'''baixar dados desde dez/2020 para que seja possivel calcular a media desde o inicio de janeiro de 2021'''
data = yf.download("petr4.SA", start="2009-12-01", end="2022-01-15")
#data


[*********************100%***********************]  1 of 1 completed


In [4]:
data_teste = data
data_aux = data.copy()

In [5]:
#obv

obv = []
obv_aux = data_teste['Volume'][0]
obv.append(obv_aux)
for i in range(1,len(data_teste)):
  #print(data_teste.index[i])
  if (data_teste['Close'][i] > data_teste['Close'][i-1]):
    obv_aux = obv_aux + data_teste['Volume'][i]
  elif (data_teste['Close'][i] < data_teste['Close'][i-1]):
    obv_aux = obv_aux - data_teste['Volume'][i]
  obv.append(obv_aux)
data_teste['obv_teste'] = obv
data_aux['obv_teste'] = obv

#data_teste.head()


In [6]:
#medias
data_teste["MME8"] = data_teste["Close"].ewm(span=8, min_periods=8).mean()
data_teste["MME20"] = data_teste["Close"].ewm(span=20, min_periods=20).mean()


In [7]:
#identificando topo no obv
topos_obv = []
sentido = []
penult_topo = data_teste["obv_teste"][0]
ultimo_topo = data_teste["obv_teste"][0]
sentido_aux = ''
for i in range(len(data_teste)-1):
  #sentido_aux = ''
  if data_teste["obv_teste"][i-1] < data_teste["obv_teste"][i] > data_teste["obv_teste"][i+1]:
    penult_topo = ultimo_topo
    ultimo_topo = data_teste["obv_teste"][i]
    if penult_topo < ultimo_topo:
      sentido_aux = 'ASC'
    elif penult_topo > ultimo_topo:
      sentido_aux = 'DESC'
    elif penult_topo == ultimo_topo:
      sentido_aux = 'LAT'
  topos_obv.append(ultimo_topo)
  sentido.append(sentido_aux)
  #print(data_teste.index[i], ultimo_topo)

for i in range(len(data_teste)-1, len(data_teste)):
  topos_obv.append(ultimo_topo)
  sentido.append(sentido_aux)

#data_teste['topos_obv'] = topos_obv
data_teste['sentido_obv'] = sentido
data_aux['sentido_obv'] = sentido

In [8]:
#Identificando topo do preço.
topos_preco = []
sentido_preco = []
penult_topo_pr = max([data_teste['Close'][0], data_teste['Open'][0]])
ultimo_topo_pr = max([data_teste['Close'][0], data_teste['Open'][0]])
sentido_aux_pr = ''
for i in range(len(data_teste)-1):
  #sentido_aux_pr = ''
  #if data_teste["Close"][i-2] < data_teste["Close"][i-1]  < data_teste["Close"][i] > data_teste["Close"][i+1]:
  consid0 = max([data_teste['Close'][i-2], data_teste['Open'][i-2]])
  consid1 = max([data_teste['Close'][i-1], data_teste['Open'][i-1]])
  consid2 = max([data_teste['Close'][i], data_teste['Open'][i]])
  consid3 = max([data_teste['Close'][i+1], data_teste['Open'][i+1]])
  

  if consid0 < consid1  < consid2 >= consid3:
    penult_topo_pr = ultimo_topo_pr
    ultimo_topo_pr = consid2 #max(abertura, fechamento)[i]
    variacao = ultimo_topo_pr / penult_topo_pr
    
    '''if penult_topo_pr < ultimo_topo_pr:
      sentido_aux_pr = 'ASC'
    elif penult_topo_pr > ultimo_topo_pr:
      sentido_aux_pr = 'DESC'
    elif penult_topo_pr == ultimo_topo_pr:
      sentido_aux_pr = 'LAT'
      '''
    #if 0.9895 < variacao < 1.0105:
    if 0.9979 < variacao < 1.0021:
      sentido_aux_pr = 'LAT'
    elif penult_topo_pr < ultimo_topo_pr and variacao > 1.0021:
      sentido_aux_pr = 'ASC'
    elif penult_topo_pr > ultimo_topo_pr and variacao < 0.9979:
      sentido_aux_pr = 'DESC'
    
  topos_preco.append(ultimo_topo_pr)
  sentido_preco.append(sentido_aux_pr)


for i in range(len(data_teste)-1, len(data_teste)):
  topos_preco.append(ultimo_topo_pr)
  sentido_preco.append(sentido_aux_pr)

data_teste['topos_preco'] = topos_preco
data_teste['sentido_topos_preco'] = sentido_preco

data_aux['topos_preco'] = topos_preco
data_aux['sentido_topos_preco'] = sentido_preco

In [9]:
#funcoes que buscam os padroes de candles
def func_j_alta(x):
  if (data_teste['Low'][x] > data_teste['High'][x-1] + 0.3 ):
    j_alta.append('sim')
  else:
    j_alta.append('não')

def func_j_baixa(x):
  if (data_teste['High'][x] < data_teste['Low'][x-1] + 0.3 ):
    #print('gap baixa')
    J_baixa.append('sim')
  else:
    #print('não')
    J_baixa.append('não')

def func_eng_alta(i):
  if(
    #PF1<PA1
    (data_teste['Close'][i-1] < data_teste['Open'][i-1])
    and 
    # PF2>PA2
    (data_teste['Close'][i] > data_teste['Open'][i])
    and
    #PF2>PA1
    (data_teste['Close'][i] > data_teste['Open'][i-1])
    and
    #PA2<PF1
    (data_teste['Open'][i]  < data_teste['Close'][i-1])
    and 
    #(PF2-PA2)/(PA1-PF1)>2
     ((data_teste['Close'][i] - data_teste['Open'][i] ) / (data_teste['Open'][i-1] - data_teste['Close'][i-1]) > 2)
    and 
    #(MA2-ME2)>(MA1-ME1)
    ((data_teste['High'][i]  - data_teste['Low'][i]) > ( data_teste['High'][i-1]  - data_teste['Low'][i-1] ))
  ):
    #print(data_teste.index[i], 'sim')
    Eng_alta.append('sim')
  else:
    #print(data_teste.index[i], 'nao')
    Eng_alta.append('não')

def func_eng_baixa(i):
  if(
     #PF1>PA1
     data_teste['Close'][i-1] > data_teste['Open'][i-1]
     and
     #PF2<PA2
     data_teste['Close'][i] < data_teste['Open'][i] 
     and
     #PF2<PA1
     data_teste['Close'][i] < data_teste['Open'][i-1]
     and
     #PA2>PF1
     data_teste['Open'][i]  > data_teste['Close'][i-1]
     and
     #(PA2-PF2)/(PF1-PA1)>2
     ((data_teste['Open'][i]  - data_teste['Close'][i]) / (data_teste['Close'][i-1] - data_teste['Open'][i-1] ) ) > 2
     and
     #(MA2-ME2)>(MA1-ME1)
     (data_teste['High'][i] - data_teste['Low'][i] ) > (data_teste['High'][i-1]  - data_teste['Low'][i-1] )
  ):
    #print(data_teste.index[i], 'sim')
    Eng_baixa.append('sim')
  else:
    #print(data_teste.index[i], 'nao')
    Eng_baixa.append('não')

def func_h_topo(i):
  if(
      #[PA1 < PF1]
     data_teste['Open'][i-1] <  data_teste['Close'][i-1]
     and 
     #[PF2 < PF1 ]
     data_teste['Close'][i] <  data_teste['Close'][i-1]
     and
     #[PA2 > PA1]
     data_teste['Open'][i]  > data_teste['Open'][i-1]
     and 
     #[(Pa2 - pf2)*3   <= (PF1 – PA1) >= (Pf2-Pa2) * 3]
     (
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) >= 3.0
     or 
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) <= -3.0
     )
  ):
    #print(data_teste.index[i], 'sim')
    Harami_topo.append('sim')
  else:
    Harami_topo.append('não')

def func_h_fundo(i):
  if(
      #[PA1 > PF1]
     data_teste['Open'][i-1] >  data_teste['Close'][i-1]
     and 
     #[PF2 > PF1 ]
     data_teste['Close'][i] >  data_teste['Close'][i-1]
     and
     #[PA2 < PA1]
     data_teste['Open'][i]  < data_teste['Open'][i-1]
     and 
     #[(Pa2 - pf2)*3   <= (PF1 – PA1) >= (Pf2-Pa2) * 3]
     (
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) >= 3.0
     or 
     ((data_teste['Close'][i-1] - data_teste['Open'][i-1]) / (data_teste['Open'][i]  - data_teste['Close'][i]) ) <= -3.0
     )
  ):
    Harami_fundo.append('sim')
  else:
    Harami_fundo.append('não')


j_alta = []
J_baixa = []
Eng_alta = []
Eng_baixa = []
Harami_topo = []
Harami_fundo = []

#verificacao
for i in range(len(data_teste)):
  #janela de alta
  #print(i)
  func_j_alta(i)
  func_j_baixa(i)
  func_eng_alta(i)
  func_eng_baixa(i)
  func_h_topo(i)
  func_h_fundo(i)



#adiciona os resultados da verificacao ao df

data_teste['J_alta'] = j_alta
data_teste['J_baixa'] = J_baixa
data_teste['Eng_alta'] = Eng_alta
data_teste['Eng_baixa'] = Eng_baixa
data_teste['Harami_topo'] = Harami_topo
data_teste['Harami_fundo'] = Harami_fundo

#data_teste


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in double_scalars


In [10]:
#Verificando coeficiente angular

coef_ang = []
tend_teste = []

for i in range(len(data_teste)):
  #x = (y2-y1) / (x2-x1)
  #y é  o preço e x é a data
  vary = data_teste['MME20'][i] - data_teste['MME20'][i-6]
  #varx = (data_teste.index[i]) - (data_teste.index[i-6])
  varx = 6 # 6 periodos
  x = vary / varx
  coef_ang.append(x)
  if  -0.1 < x < 0.1:
    tend_teste.append('TL')
  elif x <= -0.1 and data_teste['MME8'][i] < data_teste['MME20'][i]:
    tend_teste.append('TB')
  elif x >= 0.1 and data_teste['MME8'][i] > data_teste['MME20'][i]:
    tend_teste.append('TA')
  else:
    tend_teste.append('NI')
  #print('dia, ', data_teste.index[i], 'vary ', vary, ' vary/6 ', x, ' medias 8 e 20: ', data_teste['MME8'][i], data_teste['MME20'][i] )
  
data_teste['Coef_ang'] = coef_ang
data_teste['tend_medias'] = tend_teste

#data_aux['Coef_ang'] = coef_ang
data_aux['tend_medias'] = tend_teste



In [11]:
#Se sinal altista, verificar se está em tendencia de alta (através das médias), se sentido dos topos do preços = sentido topos obv = tendencia
tendencia = []
validacao = []

for i in range(len(data_teste)):
  tend = ''
  #print(data_teste.index[i], data_teste["J_alta"][i] == "sim",  data_teste["Harami_fundo"][i], data_teste["Eng_alta"][i])
  verifica = ""
  #padroes altistas
  if(data_teste["J_alta"][i] == "sim" or data_teste["Harami_fundo"][i] == "sim" or data_teste["Eng_alta"][i] == "sim"):
    tend = 'TA'
    #print(data_teste.index[i], " padrao altista", 'mme9 ', data_teste["MME9"][i], 'mme21 ', data_teste["MME21"][i])
    if(data_teste['tend_medias'][i] == "TA" and data_teste['sentido_obv'][i] == data_teste['sentido_topos_preco'][i] == 'ASC'):
      verifica = "Sim, TA"
        
  #padroes baixistas
  if(data_teste["J_baixa"][i] == "sim" or data_teste["Harami_topo"][i] == "sim" or data_teste["Eng_baixa"][i] == "sim"):
    tend = 'TB'
    #print(data_teste.index[i], " padrao altista", 'mme9 ', data_teste["MME9"][i], 'mme21 ', data_teste["MME21"][i])
    if(data_teste['tend_medias'][i] == "TB" and data_teste['sentido_obv'][i] == data_teste['sentido_topos_preco'][i] == 'DESC'):
      verifica = "Sim, tB"

  validacao.append(verifica)
  tendencia.append(tend)

#print(validacao)
data_teste["tendencia_sinal"] = tendencia
data_teste["sinal confirmado"] = validacao


data_aux["tendencia_sinal"] = tendencia
data_aux["sinal confirmado"] = validacao

In [12]:
#confirmacao do movimento
abertura = []
'''max = []
min = []'''

for i in range(len(data_teste)-1):
  abertura.append(data_teste['Open'][i+1])

for j in range(len(data_teste)-1,len(data_teste)):
  abertura.append(0)

data_teste['Conf_abert'] = abertura


#maior valor de fechamento dentro dos proximos 8periodos (B)

#porcentagem = B/A  * 100


In [13]:
menor = []
maior = []


def min(i):
  lista=[]
  menor_aux = data_teste['Low'][i]
  for j in range(i+1,i+8):
    lista.append(data_teste['Low'][j])
    if data_teste['Low'][j] < menor_aux:
      menor_aux = data_teste['Low'][j]
  menor.append(menor_aux)

def max(i):
  lista=[]
  maior_aux = data_teste['High'][i]
  for j in range(i+1,i+8):
    lista.append(data_teste['High'][j])
    if data_teste['High'][j] > maior_aux:
      maior_aux = data_teste['High'][j]
  maior.append(maior_aux)


for i in range(len(data_teste)-8):
  min(i)
  max(i)

for i in range(len(data_teste)-8, len(data_teste)):
  menor.append(0)
  maior.append(0)


data_teste['conf_min'] = menor
data_teste['conf_max'] = maior

In [14]:
percentual = []

for i in range(len(data_teste)):
  a = 0
  if data_teste['tend_medias'][i] == 'TB':
    a = 100 - (data_teste['conf_min'][i]  * 100 / data_teste['Conf_abert'][i] )
    #print('a', a)
  elif data_teste['tend_medias'][i] == 'TA':
    #print(' entrou em TA ', data_teste.index[i], data_teste['tend_medias'][i])
    a = (data_teste['conf_max'][i] * 100 / data_teste['Conf_abert'][i] ) - 100
  elif data_teste['tend_medias'][i] == 'TL':
    a = 270
  else:
    #print(' entrou no else ', data_teste.index[i], data_teste['tend_medias'][i])
    a = 250
  percentual.append(a)

data_teste['percentual'] = percentual
data_aux['percentual'] = percentual





In [17]:
#COM validacao do sinal
padroes = ['Harami_fundo',  'Harami_topo', 'J_alta', 'J_baixa', 'Eng_alta', 'Eng_baixa']
resultados = []
for i in range(len(padroes)):
  resultados = [padroes[i]]
  #FP -- qtas vezes o padrão foi formado
  mask0 = (data_teste.index >  '2010-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim')
  filtered_df = data_teste[mask0]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #VP -- qtas vezes o padrão foi validado
  mask1 = (data_teste.index >  '2010-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padroes[i]] =='sim')  & (data_teste['sinal confirmado'] != '') 
  filtered_df = data_teste[mask1]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #AM -- qtas vezes foi atingida a meta de 1% de valorização após o padrão validado
  mask2 = (data_teste.index >  '2010-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padroes[i]] =='sim')  & (data_teste['sinal confirmado'] != '') & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)
  filtered_df = data_teste[mask2]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #EF -- eficiencia
  try: 
    resultados[3] *100/resultados[2]
    resultados.append(resultados[3] *100/resultados[2])
  except:
    resultados.append(0)

  print(resultados)

['Harami_fundo', 187, 7, 5, 71.42857142857143]
['Harami_topo', 108, 2, 2, 100.0]
['J_alta', 34, 10, 10, 100.0]
['J_baixa', 824, 64, 59, 92.1875]
['Eng_alta', 64, 8, 8, 100.0]
['Eng_baixa', 78, 5, 4, 80.0]


In [18]:
#sem validacao do sinal
padroes = ['Harami_fundo',  'Harami_topo', 'J_alta', 'J_baixa', 'Eng_alta', 'Eng_baixa']
resultados = []
for i in range(len(padroes)):
  resultados = [padroes[i]]
  #FP
  #mask0 = (data_teste.index >  '2020-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim')
  mask0 = (data_teste.index >  '2010-01-01') & (data_teste.index <  '2021-12-31')& ( data_teste[padroes[i]] =='sim')
  filtered_df = data_teste[mask0]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])
  
  #AM
  mask2 = (data_teste.index >  '2010-01-01') & (data_teste.index <  '2021-12-31') & ( data_teste[padroes[i]] =='sim')  & (data_teste['percentual'] > 1) & (data_teste['percentual'] < 250)
  filtered_df = data_teste[mask2]
  resultados.append(filtered_df.iloc[: ,19:].shape[0])

  #EF
  resultados.append(resultados[2] *100/resultados[1])

  print(resultados)

['Harami_fundo', 187, 48, 25.668449197860962]
['Harami_topo', 108, 24, 22.22222222222222]
['J_alta', 34, 16, 47.05882352941177]
['J_baixa', 824, 154, 18.689320388349515]
['Eng_alta', 64, 16, 25.0]
['Eng_baixa', 78, 19, 24.358974358974358]
